In [1]:
!pip install transformers datasets


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch
import os
os.environ["WANDB_DISABLED"] = "true"
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
data = {
    "text": [
        "Once upon a time, there was a magical forest.",
        "Deep learning models are revolutionizing AI.",
        "Python is a powerful language for machine learning.",
        "Transformers work well for text generation tasks."
    ]
}

raw_dataset = Dataset.from_dict(data)


In [4]:
def tokenize_function(example):
    output = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=50
    )
    output["labels"] = output["input_ids"].copy()
    return output

tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [5]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=100,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [10]:
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch
import os
os.environ["WANDB_DISABLED"] = "true"
import pyarrow as pa

# Temporarily patch the datasets library to use np.asarray and handle ChunkedArray
def _arrow_array_to_numpy_patch(self, pa_array):
    if isinstance(pa_array, pa.ChunkedArray):
        return pa_array.to_numpy()
    array = pa_array.to_numpy_metadata()
    if (
        array.dtype == object
        or array.dtype == np.object_
        or (array.ndim > 1 and array.shape[1] > 1)
    ):
        return np.array(array, copy=False, dtype=object)
    # Use np.asarray instead of np.array(array, copy=False)
    return np.asarray(array)


# Apply the patch
from datasets.formatting import formatting
formatting.NumpyArrowExtractor._arrow_array_to_numpy = _arrow_array_to_numpy_patch


model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=6, training_loss=3.4285758336385093, metrics={'train_runtime': 70.5812, 'train_samples_per_second': 0.17, 'train_steps_per_second': 0.085, 'total_flos': 306201600000.0, 'train_loss': 3.4285758336385093, 'epoch': 3.0})

In [11]:
from transformers import pipeline

text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

output = text_generator("The future of AI is", max_length=50, num_return_sequences=1)
print(output[0]['generated_text'])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The future of AI is uncertain.


In [12]:
model.save_pretrained("my-finetuned-gpt2")
tokenizer.save_pretrained("my-finetuned-gpt2")


('my-finetuned-gpt2/tokenizer_config.json',
 'my-finetuned-gpt2/special_tokens_map.json',
 'my-finetuned-gpt2/vocab.json',
 'my-finetuned-gpt2/merges.txt',
 'my-finetuned-gpt2/added_tokens.json')